In [ ]:
# Install torch if not already installed
# %pip install torch

import torch
import torch.nn as nn
from datetime import datetime
import io
import time

model = nn.Sequential(
    nn.Linear(5, 1),
)

loss_fn = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


In [ ]:
# GET WEIGHTS FROM ALL ADDRESSES
import io
weight_ls = []
import urllib.request
contents = None
for add in address_ls:
    contents = urllib.request.urlopen("http://" + add + "/uploadweights").read()
    print(contents)
    if contents == b'shutdown':
        for add in address_ls:
            res = None
            contents = urllib.request.urlopen("http://" + add + "/searchweights").read()
            if contents == b'shutdown':
                continue
            weights = torch.load(io.BytesIO(contents))
            weight_ls.append(weights)


In [ ]:
def average_weights(w_ls):
    """
    Returns the average of the weights.
    """
    w_avg = copy.deepcopy(w_ls[0])
    for key in w_avg.keys():
        for i in range(1, len(w_ls)):
            w_avg[key] += w_ls[i][key]
        w_avg[key] = torch.div(w_avg[key], len(w_ls))
    return w_avg

address_ls = ["3.91.82.181", "3.92.27.19"]

# GET WEIGHTS FROM ALL ADDRESSES
import io
import urllib.request
import torch
import copy

weight_ls = []
contents = None
for add in address_ls:
    contents = urllib.request.urlopen("http://" + add + "/uploadweights").read()
    print(contents)
    if contents == b'shutdown':
        for add in address_ls:
            res = None
            contents = urllib.request.urlopen("http://" + add + "/searchweights").read()
            if contents == b'shutdown':
                continue
            weights = torch.load(io.BytesIO(contents))
            weight_ls.append(weights)


In [ ]:
import copy
avg_weights = average_weights(weight_ls)

model.load_state_dict(avg_weights)
model_weights = torch.save(model.state_dict(), 'avg_weights')

to_send = io.BytesIO()
torch.save(model_weights, to_send, _use_new_zipfile_serialization=False)
to_send.seek(0)


In [ ]:
import requests
for add in address_ls:
    print(add)
    print(requests.post("http://" + add + "/receiveweights", data=to_send, headers={'Content-Type': 'application/octet-stream'}))
